In [52]:
import mrigutilities as mu
import datetime 
import pandas as pd
import json
import mrigstatics

ticker = 'NIFTY 50'
data = mu.getIndexData(ticker, datetime.date.today() - datetime.timedelta(days=90), datetime.date.today())
print(max(data.index).strftime('%d-%b-%y'))

20-Jun-24


F:\Mrig Analytics\mrigAnalytics\mrigutilities.py:289: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
import sympy as sp

def evaluate_expression(expression, variable_values):
    try:
        # Parse the expression
        expr = sp.sympify(expression)
        # print(expr.free_symbols)
        
        # Substitute the variables with their values
        result = expr.subs(variable_values)
        
        return result
    except sp.SympifyError as e:
        raise ValueError(f"Error evaluating expression: {e}")

if __name__ == "__main__":
    # expression = input("Enter a mathematical expression with variables: ")
    expression = 'max(x**2 + y*(2 + x),64)'
    variable_values = {}
    variable_values['x'] = 3
    variable_values['y'] = 6
    
    # Example: To read variable values, you can do the following:
    # Assuming the variables are 'x' and 'y'
    # variable_values = {}
    # while True:
    #     var = input("Enter variable (or press Enter to finish): ")
    #     if var == "":
    #         break
    #     value = float(input(f"Enter value for {var}: "))
    #     variable_values[var] = value

    try:
        result = evaluate_expression(expression, variable_values)
        print(f"Result: {result}")
    except ValueError as e:
        print(e)


In [ ]:
# [{'symbol': 'ADANIENT', 'direction': 'short', 'instrument': 'option', 'type': 'PE', 'strike': 3050.0, 'expiry': '240627', 
#   'ltp': 49.55, 'id': 'a1'}, {'symbol': 'ADANIENT', 'direction': 'long', 'instrument': 'option', 'type': 'PE', 'strike': 3000.0, 
# 'expiry': '240627', 'ltp': 39.25, 'id': 'a2'}, {'Result': {'a1_symbol': 'ADANIENT', 'a1_direction': 'short', 'a1_instrument': 'option', 
#  'a1_type': 'PE', 'a1_strike': 3050.0, 'a1_expiry': '240627', 'a1_ltp': 49.55, 'a1_id': 'a1', 'a2_symbol': 'ADANIENT', 
# 'a2_direction': 'long', 'a2_instrument': 'option', 'a2_type': 'PE', 'a2_strike': 3000.0, 'a2_expiry': '240627', 'a2_ltp': 39.25,
# 'a2_id': 'a2', 'atm': 3243.40000000000, 'yld': 0.206000000000000, 'max_options': 5}}]

# basket
import numpy as np

import portfolios.portfolio_manager as pm
def option_strategy_analytics(basket):
    col_map = {'symbol': 'underlying','instrument':'security_type','type':'option_type','expiry':'expiry_date','tradingsymbol':'symbol'}
    sec_map = {'option' : 'OPT','futures' : 'FUT'}
    return_basket = {}
    
    for key,strategy_list in basket.items():
        return_strategy_list = []
        for strategy in strategy_list:
            portfolio = pd.DataFrame(strategy)
            portfolio.rename(columns=col_map,inplace=True)
            portfolio['security_type'] = portfolio['security_type'].map(sec_map) 
            # portfolio['Yield'] = portfolio[portfolio['underlying'].isnull()]['Yield'].head(1).values[0]
            portfolio['Yield'] = portfolio[portfolio['Yield'].notnull()]['Yield'].head(1).values[0]
            
            # print('AFTER YIELD',portfolio)            
            
            portfolio['Strategy'] = portfolio[portfolio['Strategy'].notnull()]['Strategy'].head(1).values[0]
            # print('AFTER STRATEGY',portfolio)            
            portfolio['max_profit'] = portfolio[portfolio['max_profit'].notnull()]['max_profit'].head(1).values[0]
            portfolio['max_risk'] = portfolio[portfolio['max_risk'].notnull()]['max_risk'].head(1).values[0]
            portfolio['breakeven'] = portfolio[portfolio['breakeven'].notnull()]['breakeven'].head(1).values[0]
            
            portfolio = portfolio[portfolio['underlying'].notnull()]            
            portfolio['expiry_date'] = portfolio['expiry_date'].apply(lambda x: datetime.datetime.strptime(x,'%y%m%d').date())
            # print('AFTER FINAL',portfolio)            

            portfolio = pm.run_portfolio_analytics(portfolio,spot_scenario_scale=0.1)
            return_strategy_list.append(portfolio)
        return_basket[key] = return_strategy_list

    return return_basket
        

        # {'max_profit': 1.75000000000000}, {'max_risk': 3.25000000000000}, {'breakeven': 443.250000000000}
         
    

In [ ]:
import kite.mrigkite as zkite
from itertools import permutations

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

strategy_map = { 'covered_call' : { 'trend' : 'bullish', 'objective' : 'income' } ,'short_put' : { 'trend' : 'bullish', 'objective' : 'income' } ,
                'bull_put_spread' : { 'trend' : 'bullish', 'objective' : 'income' } ,
                'bear_call_spread' : { 'trend' : 'bearish', 'objective' : 'income' } ,'long_iron_butterfly' : { 'direction' : 'neutral', 'objective' : 'income' } ,'long_iron_condor' : { 'direction' : 'neutral', 'objective' : 'income' } ,'covered_short_straddle' : { 'direction' : 'bullish', 'objective' : 'income' } ,'covered_short_strangle' : { 'direction' : 'bullish', 'objective' : 'income' } ,'calendar_call' : { 'direction' : 'bullish', 'objective' : 'income' } ,'diagonal_call' : { 'direction' : 'bullish', 'objective' : 'income' } ,'calendar_put' : { 'direction' : 'bullish', 'objective' : 'income' } ,'diagonal_put' : { 'direction' : 'bullish', 'objective' : 'income' } ,'covered_put' : { 'direction' : 'bearish', 'objective' : 'income' } ,'bull_call_spread' : { 'direction' : 'bullish', 'objective' : 'vertical' } ,'bear_put_spread' : { 'direction' : 'bearish', 'objective' : 'vertical' } ,'bull_call_ladder' : { 'direction' : 'neutral', 'objective' : 'vertical' } ,'bull_put_ladder' : { 'direction' : 'bearish', 'objective' : 'vertical' } ,'bear_call_ladder' : { 'direction' : 'bullish', 'objective' : 'vertical' } ,'bear_put_ladder' : { 'direction' : 'neutral', 'objective' : 'vertical' } ,'straddle' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'strangle' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'strip' : { 'direction' : 'bearish', 'objective' : 'volatility' } ,'strap' : { 'direction' : 'bullish', 'objective' : 'volatility' } ,'guts' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_call_butterfly' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_put_butterfly' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_call_condor' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_put_condor' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_iron_butterfly' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_iron_condor' : { 'direction' : 'neutral', 'objective' : 'volatility' } ,'short_straddle' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'short_strangle' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'short_guts' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'long_call_butterfly' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'long_put_butterfly' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'long_call_condor' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'long_put_condor' : { 'direction' : 'neutral', 'objective' : 'sideways' } ,'modified_call_butterfly' : { 'direction' : 'bullish', 'objective' : 'sideways' } ,'modified_put_butterfly' : { 'direction' : 'bullish', 'objective' : 'sideways' } ,'call_ratio_backspread' : { 'direction' : 'bullish', 'objective' : 'leveraged' } ,'put_ratio_backspread' : { 'direction' : 'bearish', 'objective' : 'leveraged' } ,'ratio_call_spread' : { 'direction' : 'bearish', 'objective' : 'leveraged' } ,'ratio_put_spread' : { 'direction' : 'bullish', 'objective' : 'leveraged' } ,'collar' : { 'direction' : 'bullish', 'objective' : 'synthetic' } ,'synthetic_call' : { 'direction' : 'bullish', 'objective' : 'synthetic' } ,'synthetic_put' : { 'direction' : 'bearish', 'objective' : 'synthetic' } ,'long_call_synthetic_straddle' : { 'direction' : '', 'objective' : 'synthetic' } ,'long_put_synthetic_straddle' : { 'direction' : '', 'objective' : 'synthetic' } ,'short_call_synthetic_straddle' : { 'direction' : '', 'objective' : 'synthetic' } ,'short_put_synthetic_straddle' : { 'direction' : '', 'objective' : 'synthetic' } ,'long_synthetic_future' : { 'direction' : 'bullish', 'objective' : 'synthetic' } ,'short_synthetic_future' : { 'direction' : 'bearish', 'objective' : 'synthetic' } ,'long_combo' : { 'direction' : 'bullish', 'objective' : 'synthetic' } ,'short_combo' : { 'direction' : 'bearish', 'objective' : 'synthetic' } ,'long_box' : { 'direction' : 'neutral', 'objective' : 'synthetic' } ,'long_call' : { 'direction' : 'bullish', 'objective' : '' } ,'long_put' : { 'direction' : 'bearish', 'objective' : '' } ,'short_call' : { 'direction' : 'bearish', 'objective' : '' } ,'long_call_synthetic' : { 'direction' : 'neutral', 'objective' : '' } ,'long_put_synthetic' : { 'direction' : 'neutral', 'objective' : '' } ,'short_call_synthetic' : { 'direction' : 'neutral', 'objective' : '' } ,'short_put_synthetic' : { 'direction' : 'neutral', 'objective' : '' }}
strategy_map = pd.DataFrame.from_dict(strategy_map,orient='index')
# strategy_map
kite_object = zkite.mrigkite()




def bull_put_spread(symbol):
    bps = {}
    bps['strategy'] = 'bps'
    bps['strategy_desc'] = 'bull_put_spread'
    bps['trend'] = ['bullish','neutral']
    bps['symbol'] = symbol
    a1 = {'symbol' : symbol,'direction':'short','instrument' : 'option','type': 'PE','strike': None,'expiry': None,'ltp': None}
    a2 = {'symbol' : symbol,'direction':'long','instrument' : 'option','type': 'PE','strike': None,'expiry': None,'ltp': None}
    bps['assets'] = {'a1':a1,'a2':a2}
    
    # Syntax for rules
    # type : asset/metric , body : expression involving variables and functions, the variables pertaining 
    # to assets should have been defined under asset json, the function used should be python functions 
    # or within the scope of strategy module
    
    r1 = {'type' : 'asset','name':'strike_rule_1','body':"a1_strike > a2_strike"}
    m1 = {'name':'atm' ,'body' : None}
    r2 = {'type' : 'asset','name':'higher_strike_rule_1','body':"a1_strike < atm"}
    # m2 = {'name':'yld','body' :" (a1_ltp - a2_ltp)/abs(a1_strike - a2_strike)"}
    r3 = {'type' : 'metric','name':'yield_rule','body': "yld > 0.10"}
    m2 = {'name':'max_options','body' : 5}
    m3 = {'name':'yield_threshold','body' : 0.1}
    m4 = {'name':'max_profit','body' : "a1_ltp - a2_ltp"}    
    m5 = {'name':'max_risk','body' : "a1_strike - a2_strike - max_profit"}
    m6 = {'name':'breakeven','body' : "a1_strike - max_profit"}
    m7 = {'name':'yld','body' : "max_profit/max_risk"}     
    
    bps['rules'] = {'r1' : r1,'r2': r2,'r3': r3}
    bps['metric'] = {'m1':m1,'m2' : m2,'m3' : m3,'m4':m4,'m5' : m5,'m6' : m6,'m7' : m7}
    
    bps = json.dumps(bps)
    
    # print(bps)
    return option_strategy(bps)

def bull_call_spread(symbol):
    bcs = {}
    bcs['strategy'] = 'bcs'
    bcs['strategy_desc'] = 'bull_call_spread'    
    bcs['trend'] = ['bullish','neutral']
    bcs['symbol'] = symbol
    a1 = {'symbol' : symbol,'direction':'short','instrument' : 'option','type': 'CE','strike': None,'expiry': None,'ltp': None}
    a2 = {'symbol' : symbol,'direction':'long','instrument' : 'option','type': 'CE','strike': None,'expiry': None,'ltp': None}
    bcs['assets'] = {'a1':a1,'a2':a2}
    
    # Syntax for rules
    # type : asset/metric , body : expression involving variables and functions, the variables pertaining 
    # to assets should have been defined under asset json, the function used should be python functions 
    # or within the scope of strategy module
    
    r1 = {'type' : 'asset','name':'strike_rule_1','body':"a1_strike > a2_strike"}
    m1 = {'name':'atm' ,'body' : None}
    r2 = {'type' : 'asset','name':'higher_strike_rule_1','body':"a2_strike > atm"}
    # m2 = {'name':'yld','body' :" (a2_ltp - a1_ltp)/abs(a1_strike - a2_strike)"}
    r3 = {'type' : 'metric','name':'yield_rule','body': "yld > 0.10"}
    m2 = {'name':'max_options','body' : 5}
    m3 = {'name':'yield_threshold','body' : 0.1}
    m4 = {'name':'max_risk','body' : "a2_ltp - a1_ltp"}
    m5 = {'name':'max_profit','body' : "a1_strike - a2_strike - (a2_ltp - a1_ltp)"}
    m6 = {'name':'breakeven','body' : "a2_strike + (a2_ltp - a1_ltp)"}
    m7 = {'name':'yld','body' : "max_profit/max_risk"}    

    
    
    bcs['rules'] = {'r1' : r1,'r2': r2,'r3': r3}
    bcs['metric'] = {'m1':m1,'m2' : m2,'m3' : m3,'m4':m4,'m5' : m5,'m6' : m6,'m7' : m7}
    
    bcs = json.dumps(bcs)
    
    # print(bcs)
    return option_strategy(bcs)

def option_strategy(strategy_json):
    os = json.loads(strategy_json)
    os
    underlying = os['symbol']
    
    # session = mu.getKiteSession()
    underlying_price = kite_object.getQuoteLive(underlying)['last_price']
    print('ATM ------->',underlying_price)
    oc = mu.kite_OC_new([underlying])
    expiry_date = min(oc['expiry'].values)
    # print(expiry_date)
    # print(oc.columns)
    assets = os['assets']
    rules = os['rules']
    metric = os['metric']
    
    for key,a in assets.items():
        a['id'] = key
    # a1 = bps['assets']['a1']
    # a1['id'] = 'a1'
    # a2 = bps['assets']['a2']
    # a2['id'] = 'a2'
    # a1_defined  = True if a1['strike'] is not None else False
    # a2_defined  = True if a2['strike'] is not None else False
    m1 = metric['m1']
    
    for key,m in metric.items():
        if 'atm' == m['name']:
            m['body'] = underlying_price
        if 'max_options' == m['name']:
            max_options = m['body']
        else:
            max_options = 5
    
    # if 'atm' == m1['name']:
        # m1['body'] = underlying_price
    # max_options = m3['max_options'] if 'max_options' in m3.keys() else 5
    oc_up_strikes = sorted(set(list(oc[(oc['strike'] >= underlying_price) & (oc['expiry'] == expiry_date)]['strike'].values)))
    # print('oc_up_strikes\n',oc_up_strikes[0:max_options])
    oc_down_strikes = sorted(set(list(oc[(oc['strike'] < underlying_price)  & (oc['expiry'] == expiry_date)]['strike'].values)))
    # print('oc_down_strikes\n',oc_down_strikes[-max_options:])
    # strikes = oc_up_strikes+oc_down_strikes
    strikes = sorted(set(oc_up_strikes+oc_down_strikes))
    # print('strikes\n',strikes)
    strikes = sorted(set(oc_up_strikes[0:max_options]+oc_down_strikes[-max_options:]))
    
    # print('strikes\n',strikes)
    
    # def get_ltp(name,strike,expiry,type):
    #     # get the instrument token
    #     ltp = oc[(oc['name'] == name) &
    #                (oc['strike'] == strike) &
    #                (oc['expiry'] == expiry) &
    #                (oc['instrument_type'] == type)]['ltp'].values
    strikes = permutations(strikes,len(assets))
    
    valid_basket = []
    for strike in list(strikes): 
        # print (strike[0],strike[1])
        asset_dict = {}
        asset_tmp = []
        i = 0
        for key, a in assets.items():
            a['strike'] = strike[i]
            a['expiry'] = expiry_date.strftime('%y%m%d')
            a['ltp'] = oc[(oc['name'] == underlying) &
                   (oc['strike'] == strike[i]) &
                   (oc['expiry'] == expiry_date) &
                   (oc['instrument_type'] == a['type'])]['last_price'].values[0]
            a['tradingsymbol'] = oc[(oc['name'] == underlying) &
                   (oc['strike'] == strike[i]) &
                   (oc['expiry'] == expiry_date) &
                   (oc['instrument_type'] == a['type'])]['tradingsymbol'].values[0]            
            asset_tmp.append(dict(a))
            
            for k in a.keys():
                asset_dict[str(a['id'])+'_'+str(k)] = a[k]   
            i = i + 1
            
        # for a in asset_tmp:
        #     for key in a.keys():
        #         asset_dict[str(a['id'])+'_'+str(key)] = a[key]
        for key,m in metric.items():
            res = evaluate_expression(m['body'], asset_dict)
            asset_dict[m['name']] =  res
            asset_tmp.append({m['name'] : res})            
        # print(asset_dict)
        rule_value = True
        for key, r in rules.items():
            expression = r['body']
            # print(expression)
            result = evaluate_expression(expression, asset_dict)
            # print(r['name']+'\n',result)    
            rule_value = rule_value & result
            # print('rule_value -> '+str(rule_value))
        if rule_value:
            # print('rule_value -> '+str(rule_value))
            # print(asset_tmp)
            # print(asset_dict)
            asset_tmp.append({'Yield' : asset_dict['yld']})
            asset_tmp.append({'Strategy' : os['strategy']})
            valid_basket.append(asset_tmp)
    
    print(valid_basket)
    return valid_basket


In [ ]:
    
symbol = 'TATAMOTORS'

symbols = mu.getIndexMembers('NIFTY 50')
print(type(symbols))
symbols = symbols.strip('][').split(', ')

symbols = [x[1:-1] for x in symbols]

basket = {}
# for s in symbols:
for s in ['TATAMOTORS']:    
    print(s)
    # try:
    a = bull_call_spread(s)
    basket[s] = a
    # except:
        # pass 
basket = option_strategy_analytics(basket)
print('basket\n\n',basket)

In [ ]:
adb = basket['TATAMOTORS'][3]
# adb
adb_a = pm.populate_portfolio_analytics(adb,db=False)
# adb_a
adb['qty'] = adb['direction'].apply(lambda x : 1 if x == 'long' else -1)
# adb
adb_a.rename(columns={'date':'valuation_date_time','secid':'security'},inplace=True)
# adb_a
adb.rename(columns={'ltp':'cost','security_type':'type','symbol':'security'},inplace=True)
# adb
adb_a = pd.merge(adb,adb_a,how='left',on='security')
# adb_a
adb_a['portfolio_name'] = adb_a['underlying']+'_'+adb_a['Strategy'].str.upper()
adb_a['strategy_id'] = adb_a['portfolio_name']+'_'+str(list(adb_a['strike']))+'_'+str(list(adb_a['expiry_date'])) 
adb_a['position_date'] = adb_a['date']
# adb_a
[port,graphs,fig,port_a] = pm.show_portfolio('any','any',portfolio=adb_a)
# port_a
# pd.DataFrame.from_dict([basket['TATAPOWER'][2]['analytics'][0]])

In [ ]:
# port_a
port_a[['strategy_id','portfolio_name','underlying','expiry_date','max_profit',	'max_risk',	'breakeven','Yield','date']].head(1)

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as subplt
import mriggraphics as mg

y_list=[]
y_names=[]
scen = pd.DataFrame()
underlying_price = port_a.iloc[0]['underlying_price']
underlying = port_a.iloc[0]['portfolio_name']
strategy = port_a.iloc[0]['Strategy'].upper()

sum_cols = []
cnt=0
for i in port_a.index:
    # print(json.loads(port_a.loc[i]['scenarios'])['SCENARIO_SPOT'])
    cost = port_a.loc[i]['cost']
    sec = port_a.loc[i]['security'] 
    qty = port_a.loc[i]['qty'] 

    # direction = port_a.loc[i]['direction']
    scen_tmp = pd.DataFrame(json.loads(port_a.loc[i]['scenarios'])['SCENARIO_SPOT'],columns=['spot','Val_'+sec])
    scen_tmp['Val_'+sec] =  scen_tmp['Val_'+sec].apply(lambda x: (x['NPV'] - cost) * qty) # (scen_tmp['Val_'+sec]['NPV'] - cost)*qty
    y_list.append(scen_tmp['Val_'+sec])
    y_names.append(sec)
    sum_cols.append('Val_'+sec)
    if cnt == 0:
        scen = scen_tmp    
    else:
        scen = scen_tmp.merge(scen,on='spot',how='inner')    
    cnt = cnt + 1

scen['spot'] = pd.to_numeric(scen['spot'])*underlying_price
scen['Val'] = scen[sum_cols].sum(axis=1)
y_list.append(scen['Val'])
y_names.append(strategy)

# cost1 = port_a.iloc[0]['cost']
# cost2 = port_a.iloc[1]['cost']
# sec1 = port_a.iloc[0]['security']
# sec2 = port_a.iloc[1]['security']

# scen1 = pd.DataFrame(json.loads(port_a.iloc[[0]]['scenarios'][0])['SCENARIO_SPOT'],columns=['spot','Val1'])
# scen2 = pd.DataFrame(json.loads(port_a.iloc[[1]]['scenarios'][1])['SCENARIO_SPOT'],columns=['spot','Val2'])
# scen = scen1.merge(scen2,on='spot',how='inner')

# scen['spot'] = pd.to_numeric(scen['spot'])*452
# scen['Val1'] = (scen['Val1'] - cost1)*-1
# scen['Val2'] = (scen['Val2'] - cost2)
# scen['Val'] = scen['Val1'] + scen['Val2']
# # scen
# # scen.plot(x='spot',grid=True)

# y_list=[scen['Val'],scen['Val1'],scen['Val2']]
# y_names=['bcs',sec1,sec2]
# sma_colors = ['purple', 'blue','darkcyan','teal']

fig = mg.plotly_line_graph(x=scen['spot'],
                          y_list=y_list,
                          y_names=y_names,
                          fig_title=underlying,
                          y_title='PnL',
                          x_title='Spot')

# fig.update_layout(yaxis_range=[-20,20])

fig.update_layout(autosize=False, width=800, height=600, showlegend=True, margin={"l":0,"r":0,"t":0.2,"b":0.2},
legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()
# port_a.iloc[[1]]['scenarios']
# pd.DataFrame([port_a.head(1)['scenarios']['SCENARIO_SPOT'].values[0]])

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as subplt
import mriggraphics as mg

y_list=[]
y_names=[]
scen = pd.DataFrame()
underlying_price = port_a.iloc[0]['underlying_price']
underlying = port_a.iloc[0]['portfolio_name']
strategy = port_a.iloc[0]['Strategy'].upper()

sum_cols = []
cnt=0
for i in port_a.index:
    # print(json.loads(port_a.loc[i]['scenarios'])['SCENARIO_SPOT'])
    cost = port_a.loc[i]['cost']
    sec = port_a.loc[i]['security'] 
    qty = port_a.loc[i]['qty'] 

    # direction = port_a.loc[i]['direction']
    scen_tmp = pd.DataFrame(json.loads(port_a.loc[i]['scenarios'])['SCENARIO_SPOT'],columns=['spot','Val_'+sec])
    scen_tmp['Val_'+sec] =  scen_tmp['Val_'+sec].apply(lambda x: (x['NPV'] - cost) * qty) # (scen_tmp['Val_'+sec]['NPV'] - cost)*qty
    y_list.append(scen_tmp['Val_'+sec])
    y_names.append(sec)
    sum_cols.append('Val_'+sec)
    if cnt == 0:
        scen = scen_tmp    
    else:
        scen = scen_tmp.merge(scen,on='spot',how='inner')    
    cnt = cnt + 1

scen['spot'] = pd.to_numeric(scen['spot'])*underlying_price
scen['Val'] = scen[sum_cols].sum(axis=1)
y_list.append(scen['Val'])
y_names.append(strategy)

# cost1 = port_a.iloc[0]['cost']
# cost2 = port_a.iloc[1]['cost']
# sec1 = port_a.iloc[0]['security']
# sec2 = port_a.iloc[1]['security']

# scen1 = pd.DataFrame(json.loads(port_a.iloc[[0]]['scenarios'][0])['SCENARIO_SPOT'],columns=['spot','Val1'])
# scen2 = pd.DataFrame(json.loads(port_a.iloc[[1]]['scenarios'][1])['SCENARIO_SPOT'],columns=['spot','Val2'])
# scen = scen1.merge(scen2,on='spot',how='inner')

# scen['spot'] = pd.to_numeric(scen['spot'])*452
# scen['Val1'] = (scen['Val1'] - cost1)*-1
# scen['Val2'] = (scen['Val2'] - cost2)
# scen['Val'] = scen['Val1'] + scen['Val2']
# # scen
# # scen.plot(x='spot',grid=True)

# y_list=[scen['Val'],scen['Val1'],scen['Val2']]
# y_names=['bcs',sec1,sec2]
# sma_colors = ['purple', 'blue','darkcyan','teal']

fig = mg.plotly_line_graph(x=scen['spot'],
                          y_list=y_list,
                          y_names=y_names,
                          fig_title=underlying,
                          y_title='PnL',
                          x_title='Spot')

# fig.update_layout(yaxis_range=[-20,20])

fig.update_layout(autosize=False, width=800, height=600, showlegend=True, margin={"l":0,"r":0,"t":0.2,"b":0.2},
legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

fig.show()
# port_a.iloc[[1]]['scenarios']
# pd.DataFrame([port_a.head(1)['scenarios']['SCENARIO_SPOT'].values[0]])

In [3]:
import research.analytics as ra
import strategies.market_instruments as mi

expiry = datetime.date(2024,6,27)
op_obj1 = mi.MarketOptions('NIFTY',23200,expiry,'CE')
op_obj2 = mi.MarketOptions('NIFTY',23400,expiry,'CE')

fut_obj = mi.MarketFutures('NIFTY',23400,expiry)
scen = ra.scenario_analysis([('NIFTY24JUN23200CE',op_obj1),('NIFTY24JUN23400CE',op_obj2),('NIFTY24MAYFUT',fut_obj)])
pd.set_option('display.max_rows', None)



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{'day_count': <QuantLib.QuantLib.Thirty360; proxy of <Swig Object of type 'QuantLib::Thirty360 *' at 0x000001B45BB51200> >, 'calendar': <QuantLib.QuantLib.India; proxy of <Swig Object of type 'QuantLib::India *' at 0x000001B45BB51080> >, 'compounding': 1, 'compounding_frequency': 1, 'interpolation': <QuantLib.QuantLib.Linear; proxy of <Swig Object of type 'Linear *' at 0x000001B45BB512C0> >, 'shiftparameter': None}


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
df_multi = {}
ss = scen.set_index('symbol')
for sym in ss.index:
    df1 = pd.DataFrame(ss.loc[sym]['SCENARIO_SPOT'],columns=['spot', 'anltk']).set_index('spot')
    df = pd.DataFrame()
    # print(df1)
    for inx in df1.index:
        # print(ss.loc[inx]['anltk'])
        df2 = pd.DataFrame(df1.loc[inx]['anltk'],index=[inx])
        # print('DF 2 \n\n',df2)
        df = pd.concat([df,df2])
        # print(df)
    df_multi[sym] = df
    # print(df_multi)
df_multi = pd.concat(df_multi, axis=1)
print(df_multi)
print(df_multi.loc[:,(slice(None),'NPV')])
# ss = ss['SCENARIO_SPOT']
# ss['NIFTY24JUN23200CE'] = pd.DataFrame(ss['NIFTY24JUN23200CE'])

# for sec in list(scen['symbol']):
#     # print(scen[scen['symbol'] == sec]['SCENARIO_SPOT'])
#     scen_spot = pd.DataFrame(scen[scen['symbol'] == sec]['SCENARIO_SPOT'].values[0], columns=['spot', sec])
#     scen_spot.set_index('spot', inplace=True)
#     # scen_spot[sec] = scen_spot[sec].apply(lambda x: pd.DataFrame(x))
#     print(scen_spot)

       NIFTY24JUN23200CE                                                  \
                     NPV     delta     gamma         theta theta_per_day   
0.9500        106.230682  0.205752  0.000304  -7099.380202    -19.450357   
0.9509        111.043953  0.212816  0.000310  -7252.729393    -19.870491   
0.9519        116.021208  0.220011  0.000316  -7404.848155    -20.287255   
0.9529        121.165460  0.227337  0.000321  -7555.538818    -20.700106   
0.9539        126.479675  0.234790  0.000327  -7704.603309    -21.108502   
0.9548        131.966761  0.242369  0.000332  -7851.843613    -21.511900   
0.9558        137.629571  0.250070  0.000337  -7997.062251    -21.909760   
0.9568        143.470894  0.257891  0.000343  -8140.062743    -22.301542   
0.9578        149.493453  0.265829  0.000348  -8280.650094    -22.686713   
0.9587        155.699898  0.273881  0.000352  -8418.631262    -23.064743   
0.9597        162.092807  0.282043  0.000357  -8553.815636    -23.435111   
0.9607      

In [ ]:
import strategies.option_strategies as os

sjson = '''{"strategy": "bcs", "strategy_desc": "bull_call_spread",
  "trend": ["bullish", "neutral"],
  "symbol": null,
  "assets": {"a1": {"symbol": null, "direction": "short", "instrument": "option", "type": "CE", "strike": null, "expiry": null, "ltp": null},
    "a2": {"symbol": null, "direction": "long", "instrument": "option", "type": "CE", "strike": null, "expiry": null, "ltp": null}},
  "rules": {"r1": {"type": "asset", "name": "strike_rule_1", "body": "a1_strike > a2_strike"},
    "r2": {"type": "asset", "name": "higher_strike_rule_1", "body": "a2_strike > atm"},
    "r3": {"type": "metric", "name": "yield_rule", "body": "yld > 0.10"}},
  "metric": {"m1": {"name": "atm", "body": null},
    "m2": {"name": "max_options", "body": 5},
    "m3": {"name": "yield_threshold", "body": 0.1},
    "m4": {"name": "max_risk", "body": "a2_ltp - a1_ltp"},
    "m5": {"name": "max_profit", "body": "a1_strike - a2_strike - (a2_ltp - a1_ltp)"},
    "m6": {"name": "breakeven", "body": "a2_strike + (a2_ltp - a1_ltp)"},
    "m7": {"name": "yld", "body": "max_profit/max_risk"}}}'''

bcs = os.OptionStrategy('BCS')
bcs.set_strategy_json(sjson)
bcs.set_symbols(['TATAMOTORS'])
bcs.run_strategy()

In [ ]:
basket = bcs.strategy_basket.copy()
basket['TATAMOTORS'][3][3]

In [ ]:
# full_list = pd.DataFrame()
mrig_engine = mu.sql_engine(dbname=mrigstatics.MRIGWEB[mrigstatics.ENVIRONMENT])
loaddate = datetime.date.today().strftime('%y%m%d')
for sym,strategylist in basket.items():
    strategylist_json = []
    for s in strategylist:
        # print(s[3].columns)
        s[3].drop(['obj','yld'],axis=1,inplace=True)
        id = s[3]['strategy_id'].head(1).values[0] 
        s[3]['valuation_date_time'] = s[3]['valuation_date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        s[3]['position_date'] = s[3]['position_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
        s[3]['date'] = s[3]['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        s[3]['expiry_date'] = s[3]['expiry_date'].astype(str) 
        s[3]['max_risk'] = pd.to_numeric(s[3]['max_risk'],errors='coerce') 
        s[3]['max_profit'] = pd.to_numeric(s[3]['max_profit'],errors='coerce') 
        s[3]['breakeven'] = pd.to_numeric(s[3]['breakeven'],errors='coerce') 
        s[3]['Yield'] = pd.to_numeric(s[3]['Yield'],errors='coerce') 
        
        
       #  s[3] = s[3][['underlying', 'direction', 'type', 'option_type', 'strike',
       # 'expiry_date', 'cost', 'id', 'security', 'atm', 'max_options',
       # 'yield_threshold','Strategy', 'underlying_price', 'cmp', 'analytics',
       # 'date', 'SCENARIO_SPOT', 'SCENARIO_TIME', 'qty', 'valuation_date_time',
       # 'price', 'delta', 'gamma', 'theta_per_day', 'vega', 'rho', 'others',
       # 'analytic_value', 'scenarios', 'portfolio_name', 'strategy_id',
       # 'position_date']]
        sql = "insert into os_page (strategy_id, strategy_obj,load_date, strategy_table, strategy_name) values (%s,%s,%s,'','')"
        # strategylist_json.append(json.dumps([s[3].to_dict(),s[1]]))   
        mrig_engine.execute(sql,(id,json.dumps([s[3].to_dict(),s[1]]),loaddate))
#         print(full_list)
# strategylist_json[3]

In [9]:
id = '1xWWiVRS6iNy4ghyvsLt1718793986_748982'
mrig_engine = mu.sql_engine(dbname=mrigstatics.MRIGWEB[mrigstatics.ENVIRONMENT])

strategy = mrig_engine.execute("select strategy_obj from os_page where strategy_id='"+id+"'").fetchall()[0][0]
# print(strategy)

strategy = json.loads(strategy)
df = strategy[0]
graphs = strategy[1]
df = pd.DataFrame(df)

df.columns

Index(['underlying', 'direction', 'type', 'option_type', 'strike',
       'expiry_date', 'cost', 'id', 'security', 'atm', 'max_options',
       'yield_threshold', 'max_profit', 'max_risk', 'breakeven', 'Yield',
       'Strategy', 'underlying_price', 'cmp', 'analytics', 'date',
       'SCENARIO_SPOT', 'SCENARIO_TIME', 'qty', 'valuation_date_time', 'price',
       'delta', 'gamma', 'theta_per_day', 'vega', 'rho', 'others',
       'analytic_value', 'scenarios', 'portfolio_name', 'strategy_id',
       'position_date'],
      dtype='object')

In [33]:
mrig_engine = mu.sql_engine(dbname=mrigstatics.MRIGWEB[mrigstatics.ENVIRONMENT])
strategy_basket = mrig_engine.execute("select strategy_id, strategy_obj from os_page").fetchall()
strategy_basket# strategy_basket = json.loads(strategy_basket)
# pd.DataFrame(strategy_basket[0])

[('HlmmFEFBiscZ0PLKHUHS1718882019_069989', '[{"underlying": {"0": "TATAMOTORS", "1": "TATAMOTORS"}, "direction": {"0": "short", "1": "long"}, "type": {"0": "OPT", "1": "OPT"}, "option_type": {" ... (11631688 characters truncated) ... lue\\"}},                        {\\"responsive\\": true}                    )                };                            </script>        </div>"]'),
 ('fWD947m7sAYeBzSLTBl91718882019_829244', '[{"underlying": {"0": "SBIN", "1": "SBIN"}, "direction": {"0": "short", "1": "long"}, "type": {"0": "OPT", "1": "OPT"}, "option_type": {"0": "PE", "1 ... (11590513 characters truncated) ... lue\\"}},                        {\\"responsive\\": true}                    )                };                            </script>        </div>"]'),
 ('qoU2ox8Gwm6U83hIf3eZ1718882020_208555', '[{"underlying": {"0": "HDFCBANK", "1": "HDFCBANK"}, "direction": {"0": "short", "1": "long"}, "type": {"0": "OPT", "1": "OPT"}, "option_type": {"0":  ... (11504092 characters truncat

In [77]:
import research.analytics as ra
file = 'strategies\option_strategies.json'
f = open(file)
f = json.load(f)
f= pd.DataFrame.from_dict(f['strategy_map'])
stratlist = f[(f['implementation'] == 'True')].index.to_list()
for strat in stratlist:
    print(f[strat])
# df = ra.display_tech_analysis()
# print(df[df['symbol'] == 'HDFCBANK'])

KeyError: 'bull_put_spread'

In [75]:
engine = mu.sql_engine()
df = pd.read_sql("select * from technicals where date = (select max(date) from technicals)",engine)
# df.set_index('symbol',inplace=True)
df['ta'] = df['ta'].apply(lambda x: json.loads(x))
df1 = pd.json_normalize(df['ta'])
df = df.join(df1)
df[df['SUPERTd_7_3.0'] == -1]

,symbol,ta,date,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
0,ABB,"{'MACD_12_26_9': 289.60265851528675, 'MACDh_12...",2024-06-20,289.602659,-5.991133,295.593791,9030.296543,-1.0,NaN,9030.296543
1,ADANIENSOL,"{'MACD_12_26_9': -11.665115378397559, 'MACDh_1...",2024-06-20,-11.665115,-3.757699,-7.907416,1144.101673,-1.0,NaN,1144.101673
2,ADANIENT,"{'MACD_12_26_9': 37.05942042801735, 'MACDh_12_...",2024-06-20,37.059420,-2.135879,39.195299,3629.618902,-1.0,NaN,3629.618902
3,ADANIGREEN,"{'MACD_12_26_9': -8.587364667178235, 'MACDh_12...",2024-06-20,-8.587365,-8.722328,0.134963,2022.571578,-1.0,NaN,2022.571578
4,ADANIPORTS,"{'MACD_12_26_9': 24.184202837489238, 'MACDh_12...",2024-06-20,24.184203,5.194010,18.990193,1607.296742,-1.0,NaN,1607.296742
6,AMBUJACEM,"{'MACD_12_26_9': 13.243957906969172, 'MACDh_12...",2024-06-20,13.243958,3.947378,9.296580,699.388155,-1.0,NaN,699.388155
8,ASIANPAINT,"{'MACD_12_26_9': 11.81934617986417, 'MACDh_12_...",2024-06-20,11.819346,-0.538490,12.357836,2976.896659,-1.0,NaN,2976.896659
9,ATGL,"{'MACD_12_26_9': -1.3804316782017167, 'MACDh_1...",2024-06-20,-1.380432,-6.493001,5.112569,1047.581011,-1.0,NaN,1047.581011
12,BAJAJFINSV,"{'MACD_12_26_9': -0.06093317861132164, 'MACDh_...",2024-06-20,-0.060933,6.940029,-7.000962,1631.950476,-1.0,NaN,1631.950476
15,BANKBARODA,"{'MACD_12_26_9': 5.342558943546351, 'MACDh_12_...",2024-06-20,5.342559,1.637669,3.704890,306.580485,-1.0,NaN,306.580485
